# Lecture 18
## Wednesday, November 8th, 2017
## Databases with `SQlite`

# Exercises
Have them do an exercise with the candidates and contributors dataset.

They should create a database with that dataset (like in Lecture 17).

Then show them a `pandas` command to write pretty tables.

Show them some more commands:
* Various query commands
* Sorting commands
* SELECT-COLUMNS and SELECT-DISTINCT
* Have them update the tables with new records
* AGGREGATE?
* DELETE
* LIMIT
* INDEXES

Then get them to do some joins:
* implicit join
* explicit inner join
* outer join
* right outer join
* full join

Might want to explain joins a bit before actually doing them.

In [1]:
import sqlite3

In [2]:
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

In [3]:
db = sqlite3.connect('L18DB.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

In [4]:
cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

In [5]:
cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

In [6]:
with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

In [7]:
with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

In [8]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
q = cursor.execute('''SELECT * FROM candidates''')
q = q.fetchall()
framelist = []
for i, col_name in enumerate(candidate_cols):
    framelist.append((col_name, [col[i] for col in q]))
pd.DataFrame.from_items(framelist)

,id,first_name,last_name,middle_init,party
0,16,Mike,Huckabee,,R
1,20,Barack,Obama,,D
2,22,Rudolph,Giuliani,,R
3,24,Mike,Gravel,,D
4,26,John,Edwards,,D
5,29,Bill,Richardson,,D
6,30,Duncan,Hunter,,R
7,31,Dennis,Kucinich,,D
8,32,Ron,Paul,,R
9,33,Joseph,Biden,,D


In [9]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
q = cursor.execute('''SELECT * FROM contributors''')
q = q.fetchall()
framelist = []
for i, col_name in enumerate(contributor_cols):
    framelist.append((col_name, [col[i] for col in q]))
pd.DataFrame.from_items(framelist)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16
5,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
6,7,Akin,Rebecca,,181 Baywood Lane,,Monticello,AR,71655,500.0,2007-05-18,16
7,8,Aldridge,Brittni,,"808 Capitol Square Place, SW",,Washington,DC,20024,250.0,2007-06-06,16
8,9,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1000.0,2007-06-11,16
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16


In [10]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

query = '''SELECT * FROM candidates WHERE middle_init <> ""'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,34,Hillary,Clinton,R.,D
1,39,Christopher,Dodd,J.,D
2,41,Fred,Thompson,D.,R


In [11]:
query = '''SELECT * FROM contributors WHERE state="PA"'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
1,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,94,Raught,Philip,M,4714 Plum Way,,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
3,95,Ferrara,Judith,D,1508 Waterford Road,,Yardley,PA,19067,-1100.0,2008-04-21,32
4,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


In [12]:
query = '''SELECT * FROM contributors WHERE state="PA" AND city="BETHLEHEM"'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
1,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


In [13]:
query = '''SELECT * FROM contributors WHERE amount>1000.00'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
1,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
7,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35


In [14]:
query = '''SELECT * FROM contributors WHERE state =""'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,,LAGE KAART 77,,BRASSCHATT,,2930,-500.0,2008-11-20,35


In [15]:
query = '''SELECT * FROM contributors WHERE state IN ('VA', 'WA')'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.00,2007-06-30,16
1,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.00,2007-09-19,20
2,63,BURKE,SUZANNE,M.,3401 EVANSTON,,SEATTLE,WA,981038677,-700.00,2008-03-05,22
3,78,Ranganath,Anoop,,2507 Willard Drive,,Charlottesville,VA,22903,-100.00,2008-04-21,32
4,89,Perreault,Louise,,503 Brockridge Hunt Drive,,Hampton,VA,23666,-34.08,2008-04-21,32
5,101,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.00,2008-02-08,34
6,107,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.00,2008-02-14,34
7,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [16]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.00 and 5000.00'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
1,9,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1000.0,2007-06-11,16
2,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
3,11,Allison,John W.,,P.O. Box 1089,,Conway,AR,72033,1000.0,2007-05-18,16
4,12,Allison,Rebecca,,3206 Summit Court,,Little Rock,AR,72227,1000.0,2007-04-25,16
5,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
6,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
7,20,Atiq,Omar,,7200 S Hazel Street,,Pine Bluff,AR,71603,1000.0,2007-05-18,16
8,21,Atiq,Omar,,7200 S Hazel Street,,Pine Bluff,AR,71603,1000.0,2007-06-27,16
9,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16


In [17]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.00 and 5000.00 ORDER BY amount DESC'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
7,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
8,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16


In [18]:
query = '''SELECT * FROM candidates ORDER BY last_name'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,33,Joseph,Biden,,D
1,36,Samuel,Brownback,,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,,D
5,22,Rudolph,Giuliani,,R
6,24,Mike,Gravel,,D
7,16,Mike,Huckabee,,R
8,30,Duncan,Hunter,,R
9,31,Dennis,Kucinich,,D


In [19]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.00 AND 5000.00 ORDER BY candidate_id, amount DESC'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
1,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
2,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
3,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
4,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
5,9,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1000.0,2007-06-11,16
6,11,Allison,John W.,,P.O. Box 1089,,Conway,AR,72033,1000.0,2007-05-18,16
7,12,Allison,Rebecca,,3206 Summit Court,,Little Rock,AR,72227,1000.0,2007-04-25,16
8,20,Atiq,Omar,,7200 S Hazel Street,,Pine Bluff,AR,71603,1000.0,2007-05-18,16
9,21,Atiq,Omar,,7200 S Hazel Street,,Pine Bluff,AR,71603,1000.0,2007-06-27,16


In [20]:
query = '''SELECT DISTINCT last_name, first_name FROM contributors'''
viz_tables(['last_name', 'first_name'], query)

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
2,Akin,Charles
3,Akin,Mike
4,Akin,Rebecca
5,Aldridge,Brittni
6,Allen,John D.
7,Allison,John W.
8,Allison,Rebecca
9,Altes,R.D.


In [21]:
cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')

In [22]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
viz_tables(candidate_cols, '''SELECT * FROM candidates''')

,id,first_name,last_name,middle_init,party,full_name
0,16,Mike,Huckabee,,R,None
1,20,Barack,Obama,,D,None
2,22,Rudolph,Giuliani,,R,None
3,24,Mike,Gravel,,D,None
4,26,John,Edwards,,D,None
5,29,Bill,Richardson,,D,None
6,30,Duncan,Hunter,,R,None
7,31,Dennis,Kucinich,,D,None
8,32,Ron,Paul,,R,None
9,33,Joseph,Biden,,D,None


In [23]:
max_donated = cursor.execute("SELECT *, MAX(amount) AS max_amt FROM contributors")
viz_tables(contributor_cols, "SELECT *, MAX(amount) AS max_amt FROM contributors")

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20


In [24]:
how_many = cursor.execute("SELECT COUNT(amount) AS num_above FROM contributors WHERE amount > 1000")
print(how_many.fetchall())
#print(how_many.fetchall()[0][0])

[(12,)]


In [25]:
function = '''SELECT state, AVG(amount) FROM contributors GROUP BY state'''
viz_tables(['state', 'sum'], function)

,state,sum
0,,-500.000000
1,AK,403.333333
2,AR,1183.333333
3,AZ,120.000000
4,CA,-217.988261
5,CO,-1455.750000
6,CT,2300.000000
7,DC,-309.982000
8,FL,-135.000000
9,IA,250.000000


In [26]:
deletion = '''DELETE FROM contributors WHERE last_name=\"Ahrens\"'''
query = '''SELECT * FROM contributors'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party,full_name
0,1,Agee,Steven,,549 Laurel Branch Road,
1,2,Ahrens,Don,,4034 Rennellwood Way,
2,3,Ahrens,Don,,4034 Rennellwood Way,
3,4,Ahrens,Don,,4034 Rennellwood Way,
4,5,Akin,Charles,,10187 Sugar Creek Road,
5,6,Akin,Mike,,181 Baywood Lane,
6,7,Akin,Rebecca,,181 Baywood Lane,
7,8,Aldridge,Brittni,,"808 Capitol Square Place, SW",
8,9,Allen,John D.,,1052 Cannon Mill Drive,
9,10,Allen,John D.,,1052 Cannon Mill Drive,


In [27]:
query = '''SELECT * FROM contributors ORDER BY amount DESC LIMIT 10'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
7,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
8,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16


In [28]:
query = '''SELECT * FROM contributors WHERE amount > 0 ORDER BY amount LIMIT 10'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.0,2007-08-16,20
1,50,Harrison,Ryan,,2247 3rd St,,La Verne,CA,917504918,25.0,2007-07-26,20
2,102,Aarons,Elaine,,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
3,141,ABEGG,PATRICIA,T.,1862 E. 5150 S.,,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
4,144,ABEGG,PATRICIA,T.,1862 E. 5150 S.,,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
5,145,ABEGG,PATRICIA,T.,1862 E. 5150 S.,,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
6,161,ABAIR,PETER,,40 EVANS STREET,,WATERTOWN,MA,24722150,25.0,2008-01-09,37
7,159,ABBO,PAULINE,MORENCY,10720 JACOB LANE,,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
8,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16
9,19,Ardle,William,,412 Dakota Avenue,,Springfield,OH,45504,50.0,2007-06-28,16


In [29]:
query = '''SELECT * FROM contributors '''

In [30]:
cols = ["contributors.last_name", "contributors.first_name", "contributors.amount", "candidates.last_name"]
query = '''SELECT contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name FROM contributors, candidates WHERE contributors.candidate_id = candidates.id AND candidates.last_name = "Obama"'''
viz_tables(cols, query)

,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Buckler,Steve,50.00,Obama
1,Buckler,Steve,25.00,Obama
2,Buckheit,Bruce,100.00,Obama
3,Buckel,Linda,2300.00,Obama
4,Buckel,Linda,-2300.00,Obama
5,Buckel,Linda,4600.00,Obama
6,Buck,Thomas,100.00,Obama
7,Buck,Jay,200.00,Obama
8,Buck,Blaine,2300.00,Obama
9,Buck,Barbara,50.00,Obama


In [31]:
cols = ["contributors.last_name", "contributors.first_name", "contributors.amount", "candidates.last_name"]
query = '''SELECT contributors.last_name, contributors.first_name, amount, candidates.last_name FROM contributors INNER JOIN candidates ON candidates.id = contributors.candidate_id'''
viz_tables(cols, query)


,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Agee,Steven,500.0,Huckabee
1,Ahrens,Don,250.0,Huckabee
2,Ahrens,Don,50.0,Huckabee
3,Ahrens,Don,100.0,Huckabee
4,Akin,Charles,100.0,Huckabee
5,Akin,Mike,1500.0,Huckabee
6,Akin,Rebecca,500.0,Huckabee
7,Aldridge,Brittni,250.0,Huckabee
8,Allen,John D.,1000.0,Huckabee
9,Allen,John D.,1300.0,Huckabee


In [32]:
cols = ["contributors.last_name", "contributors.first_name", "contributors.amount", "candidates.last_name"]
query = '''SELECT contributors.last_name, contributors.first_name, amount, candidates.last_name FROM contributors INNER JOIN candidates ON candidates.id = contributors.candidate_id WHERE candidates.last_name = "Obama"'''
viz_tables(cols, query)

,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Buckler,Steve,50.00,Obama
1,Buckler,Steve,25.00,Obama
2,Buckheit,Bruce,100.00,Obama
3,Buckel,Linda,2300.00,Obama
4,Buckel,Linda,-2300.00,Obama
5,Buckel,Linda,4600.00,Obama
6,Buck,Thomas,100.00,Obama
7,Buck,Jay,200.00,Obama
8,Buck,Blaine,2300.00,Obama
9,Buck,Barbara,50.00,Obama


In [33]:
q = '''SELECT contributors.last_name, contributors.first_name, amount, candidates.last_name FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id'''
viz_tables(["contributors.last_name", "contributors.first_name", "amount", "candidates.last_name"], q)

,contributors.last_name,contributors.first_name,amount,candidates.last_name
0,Agee,Steven,500.0,Huckabee
1,Ahrens,Don,250.0,Huckabee
2,Ahrens,Don,50.0,Huckabee
3,Ahrens,Don,100.0,Huckabee
4,Akin,Charles,100.0,Huckabee
5,Akin,Mike,1500.0,Huckabee
6,Akin,Rebecca,500.0,Huckabee
7,Aldridge,Brittni,250.0,Huckabee
8,Allen,John D.,1000.0,Huckabee
9,Allen,John D.,1300.0,Huckabee


In [34]:
q = '''SELECT contributors.last_name, contributors.first_name, amount, candidates.last_name FROM contributors INNER JOIN candidates ON contributors.candidate_id = candidates.id WHERE candidates.last_name = "Obama"'''
viz_tables(["contributors.last_name", "contributors.first_name", "amount", "candidates.last_name"], q)

,contributors.last_name,contributors.first_name,amount,candidates.last_name
0,Buckler,Steve,50.00,Obama
1,Buckler,Steve,25.00,Obama
2,Buckheit,Bruce,100.00,Obama
3,Buckel,Linda,2300.00,Obama
4,Buckel,Linda,-2300.00,Obama
5,Buckel,Linda,4600.00,Obama
6,Buck,Thomas,100.00,Obama
7,Buck,Jay,200.00,Obama
8,Buck,Blaine,2300.00,Obama
9,Buck,Barbara,50.00,Obama
